# AI Comic Book Generator - Colab Launcher

This notebook runs the AI Comic Generator **completely FREE** using local Ollama models.
**No API keys needed!** Everything runs on Colab's free GPU.

In [ ]:
# @title 1. Initial Setup (Run Once)
# @markdown Installs Ollama, zstd, and clones the repository.
REPO_URL = "https://github.com/yogesh-dixit-dev/comic-generator.git" # @param {type:"string"}
BRANCH = "master" # @param {type:"string"}

import os
import sys

# Install zstd (required by Ollama install script)
print("📦 Installing dependencies (zstd)...")
!apt-get update -qq && apt-get install -y -qq zstd

# Install Ollama
print("📦 Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama server in background
print("🚀 Starting Ollama server...")
!nohup ollama serve > ollama.log 2>&1 &

# Pull the models
import time
time.sleep(5) # Wait for server
print("⬇️ Downloading Reasoning Model (Llama 3.1 8B - this may take 5-10 mins)...")
!ollama pull llama3.1:8b

print("⬇️ Downloading Fast Model (Llama 3.2 3B)...")
!ollama pull llama3.2:3b

# Clone or update repo
if not os.path.exists("comic-gen"):
    !git clone -b $BRANCH $REPO_URL comic-gen

%cd /content/comic-gen
!pip install -q -r requirements.txt

print("\n✅ Core Installation Complete!")

In [ ]:
# @title 2. Update Code (Run after every fix!)
# @markdown Pulls the latest changes from GitHub WITHOUT re-installing everything.

%cd /content/comic-gen
print("🔄 Pulling latest code...")
!git fetch --all
!git reset --hard origin/master
print("\n✅ Code Updated to Latest Version!")

In [ ]:
# @title 3. Configuration (Optional - Hugging Face Storage)
import os
from google.colab import userdata

try:
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    print("✅ Loaded HF token from Colab Secrets")
except:
    print("⚠️ No HF_TOKEN in Colab Secrets. Output will stay local.")
    pass

In [ ]:
# @title 4. Run Pipeline (Tiered Local Models - No API Calls!)
INPUT_FILE = "story.txt" # @param {type:"string"}
OUTPUT_DIR = "output" # @param {type:"string"}
USE_HF_STORAGE = False # @param {type:"boolean"}
HF_REPO_ID = "your-username/comic-dataset" # @param {type:"string"}

import os
os.chdir('/content/comic-gen')
    
print(f"📂 Working directory: {os.getcwd()}")
print("🤖 Using TIERED Local Models (Llama 3.1 8B + Llama 3.2 3B) - No API calls!\n")

storage_flag = f"--storage hf --hf_repo '{HF_REPO_ID}'" if USE_HF_STORAGE and os.environ.get('HF_TOKEN') else "--storage local"

!python src/main.py --input "$INPUT_FILE" \
                    --output "$OUTPUT_DIR" \
                    --reasoning_model "ollama/llama3.1:8b" \
                    --fast_model "ollama/llama3.2:3b" \
                    --colab {storage_flag}

In [ ]:
# @title 5. Download Output
from google.colab import files
import os

if os.path.exists('output'):
    !zip -r comic_output.zip output
    files.download('comic_output.zip')
    print("✅ Download started!")
else:
    print("⚠️ No output folder found. Did the pipeline run successfully?")